In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy import signal
from scipy.ndimage import median_filter
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load the data
train_df = pd.read_csv("hacktrain.csv")
test_df = pd.read_csv("hacktest.csv")

In [3]:
# Clean column names
if 'Unnamed: 0' in train_df.columns:
    train_df = train_df.drop("Unnamed: 0", axis=1)
if 'Unnamed: 0' in test_df.columns:
    test_df = test_df.drop("Unnamed: 0", axis=1)


In [4]:
feature_cols = [col for col in train_df.columns if col not in ['ID', 'class']]
print(f"Found {len(feature_cols)} NDVI time series features")

Found 27 NDVI time series features


In [5]:
# Prepare training data
X_train_full = train_df[feature_cols]
y_train_full = train_df['class']
X_test = test_df[feature_cols]

In [6]:
#  DENOISING TECHNIQUES
def advanced_denoising(df, feature_cols):
    """Apply multiple denoising techniques for robust noise reduction"""
    df_denoised = df.copy()
    
    # 1. Median filter for outlier removal (works well for salt-and-pepper noise)
    for col in feature_cols:
        # Apply median filter with window size 3
        df_denoised[col] = median_filter(df[col].fillna(df[col].median()), size=3)
    
    # 2. Savitzky-Golay filter for temporal smoothing
    for idx in df_denoised.index:
        row_values = df_denoised.loc[idx, feature_cols].values
        if not np.isnan(row_values).all():
            # Fill NaN values temporarily for filtering
            row_filled = pd.Series(row_values).fillna(method='ffill').fillna(method='bfill').values
            try:
                # Apply Savitzky-Golay filter (polynomial order 2, window length 5)
                if len(row_filled) >= 5:
                    smoothed = signal.savgol_filter(row_filled, window_length=5, polyorder=2)
                    df_denoised.loc[idx, feature_cols] = smoothed
            except:
                pass  # Keep original values if filtering fails
    
    return df_denoised

# Apply advanced denoising
X_train_denoised = advanced_denoising(X_train_full, feature_cols)
X_test_denoised = advanced_denoising(X_test, feature_cols)


In [7]:
#  imputation 
# Use median for robustness against outliers
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_denoised), 
                              columns=feature_cols, index=X_train_denoised.index)
X_test_imputed = pd.DataFrame(imputer.transform(X_test_denoised), 
                             columns=feature_cols, index=X_test_denoised.index)

print("Advanced denoising and imputation complete.")


Advanced denoising and imputation complete.


In [9]:
# Feature Enginnering
def create_comprehensive_features(df, feature_cols):
    """Create comprehensive features for temporal NDVI analysis"""
    df_featured = df.copy()
    
    # Basic statistical features
    df_featured['ndvi_mean'] = df[feature_cols].mean(axis=1)
    df_featured['ndvi_std'] = df[feature_cols].std(axis=1)
    df_featured['ndvi_max'] = df[feature_cols].max(axis=1)
    df_featured['ndvi_min'] = df[feature_cols].min(axis=1)
    df_featured['ndvi_range'] = df_featured['ndvi_max'] - df_featured['ndvi_min']
    df_featured['ndvi_median'] = df[feature_cols].median(axis=1)
    
    # Advanced statistical features
    df_featured['ndvi_q25'] = df[feature_cols].quantile(0.25, axis=1)
    df_featured['ndvi_q75'] = df[feature_cols].quantile(0.75, axis=1)
    df_featured['ndvi_iqr'] = df_featured['ndvi_q75'] - df_featured['ndvi_q25']
    df_featured['ndvi_skew'] = df[feature_cols].skew(axis=1)
    df_featured['ndvi_kurt'] = df[feature_cols].kurtosis(axis=1)
    
    # Temporal trend features
    time_series_data = df[feature_cols].values
    
    # Linear trend slope
    trends = []
    for i in range(len(time_series_data)):
        x = np.arange(len(feature_cols))
        y = time_series_data[i]
        valid_idx = ~np.isnan(y)
        if np.sum(valid_idx) > 1:
            slope = np.polyfit(x[valid_idx], y[valid_idx], 1)[0]
        else:
            slope = 0
        trends.append(slope)
    df_featured['ndvi_trend'] = trends
    
    # Seasonal variation (assuming roughly monthly data)
    seasonal_std = []
    for i in range(len(time_series_data)):
        y = time_series_data[i]
        valid_y = y[~np.isnan(y)]
        if len(valid_y) > 3:
            # Calculate seasonal variation as std of detrended series
            if len(valid_y) >= 4:
                detrended = signal.detrend(valid_y)
                seasonal_std.append(np.std(detrended))
            else:
                seasonal_std.append(np.std(valid_y))
        else:
            seasonal_std.append(0)
    df_featured['ndvi_seasonal_var'] = seasonal_std
    
    # Vegetation vigor features (specific to NDVI)
    df_featured['ndvi_vigor'] = df_featured['ndvi_mean'] * (1 - df_featured['ndvi_std'])
    df_featured['ndvi_stability'] = 1 / (1 + df_featured['ndvi_std'])
    
    # Peak detection features
    peak_counts = []
    for i in range(len(time_series_data)):
        y = time_series_data[i]
        valid_y = y[~np.isnan(y)]
        if len(valid_y) > 5:
            peaks, _ = signal.find_peaks(valid_y, height=np.mean(valid_y))
            peak_counts.append(len(peaks))
        else:
            peak_counts.append(0)
    df_featured['ndvi_peak_count'] = peak_counts
    
    return df_featured

# Apply comprehensive feature engineering
X_train_featured = create_comprehensive_features(X_train_imputed, feature_cols)
X_test_featured = create_comprehensive_features(X_test_imputed, feature_cols)

print(f"Feature engineering complete. Total features: {X_train_featured.shape[1]}")


Feature engineering complete. Total features: 43


In [10]:
# Feature Scaling and Encoding
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_featured)
X_test_scaled = scaler.transform(X_test_featured)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_full)


In [11]:
# Model Training
final_model = LogisticRegression(
    solver='liblinear',
    multi_class='ovr',
    C=1.0,  # Slightly reduced regularization for better generalization
    penalty='l1',  # L1 regularization for feature selection
    random_state=42,
    class_weight='balanced',  # Handle class imbalance
    max_iter=2000,  # Increased iterations for convergence
    tol=1e-6  # Tighter tolerance for better convergence
)

# Train on full training data
final_model.fit(X_train_scaled, y_train_encoded)

print("Model training complete.")


Model training complete.


In [12]:
# Cross validation assessment
# Perform stratified k-fold cross-validation on training data only
cv_scores = cross_val_score(
    final_model, X_train_scaled, y_train_encoded, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy'
)

print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

Cross-validation scores: [0.91125  0.910625 0.91125  0.91875  0.9125  ]
Mean CV accuracy: 0.9129 (+/- 0.0060)


In [13]:
# Make predictions test set
final_predictions_encoded = final_model.predict(X_test_scaled)
final_predictions = label_encoder.inverse_transform(final_predictions_encoded)

# Create submission file
final_submission_df = pd.DataFrame({
    'ID': test_df['ID'], 
    'class': final_predictions
})
final_submission_df.to_csv('enhanced_final_submission3.csv', index=False)

print(f"Predictions generated for all {len(test_df)} samples in hacktest.csv")

Predictions generated for all 2845 samples in hacktest.csv


In [14]:
# Prediction confidenece
# Get prediction probabilities for the test set
prediction_probs = final_model.predict_proba(X_test_scaled)
max_probs = np.max(prediction_probs, axis=1)

print(f"Prediction confidence statistics:")
print(f"Mean confidence: {max_probs.mean():.4f}")
print(f"Min confidence: {max_probs.min():.4f}")
print(f"Max confidence: {max_probs.max():.4f}")
print(f"Std confidence: {max_probs.std():.4f}")

# Count predictions by class
class_counts = pd.Series(final_predictions).value_counts()
print(f"\nPrediction distribution on hacktest.csv:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count} ({count/len(final_predictions)*100:.1f}%)")


Prediction confidence statistics:
Mean confidence: 0.8009
Min confidence: 0.2759
Max confidence: 1.0000
Std confidence: 0.2005

Prediction distribution on hacktest.csv:
forest: 1377 (48.4%)
farm: 628 (22.1%)
impervious: 397 (14.0%)
grass: 252 (8.9%)
water: 126 (4.4%)
orchard: 65 (2.3%)
